# Emsemble Recommender

V1 - Jun 26th

This is the final emsemble notebook to join several partial solutions in one.
It's a very basic approach but it showed good results.
Mainly we reload solutions files generated by different ways and join them into weighted one:
 - Weights come in the shape of %
 - When a partial solution is unable to meet its % quota we tried to get more data from other solutions if possible 

### Load solution file from CB Recommender

In [1]:
import pandas as pd

solution_cbf=pd.read_csv('14227_cb_solution_Jun27_055_vfull300.csv',names=['item_id','users'],sep='\t',header=None)
solution_cbf['dataset']='CB'

print solution_cbf.shape
solution_cbf.head()

(46442, 3)


,item_id,users,dataset
0,103,"968548,1121184,240124,169200,590618,709594,202...",CB
1,113,1235656,CB
2,260,"865895,2109770,865895,878845,311024,684334,865...",CB
3,289,"1480022,1480022,2253016,2253016,968984,408773,...",CB
4,300,"542043,865895,1741772,686307,1035963,191061,12...",CB


### Load solution from Interaction based regressor/classifier

In [2]:
solution_int=pd.read_csv('13271_250solution.csv',names=['item_id','users'],sep='\t',header=None)
solution_int['dataset']='INT'

print solution_int.shape
solution_int.head()

(45011, 3)


,item_id,users,dataset
0,103,"2270030,427739,259811,2053817,1948765,242845,3...",INT
1,113,"1480056,105771,830673,334114,1481602,1438624,3...",INT
2,260,"565375,2233811,1785365,231060,564809,1164291,2...",INT
3,289,"2253016,1961197,1564852,1828191,2327258,440825...",INT
4,300,"285760,1943554,675261,1833566,139926,830925,17...",INT


### We also load Recsys baseline to optional add 0% to 1% of diversity

In [4]:
solution_base=pd.read_csv('solution_file_example.csv',names=['item_id','users'],sep='\t',header=None)
solution_base['dataset']='BASE'

print solution_base.shape
solution_base.head()

(44905, 3)


,item_id,users,dataset
0,103,"64851,99374,117614,280728,590206,719974,112857...",BASE
1,113,"609021,2301975,544185,193433,1015484,1480056,1...",BASE
2,260,"546777,246835,522775,646282,734332,917179,1231...",BASE
3,289,"206043,1216259,1760971,229639,438448,708718,87...",BASE
4,300,"613985,646649,102990,220473,314643,374980,8344...",BASE


### Perform a joining of solution based on quota-% and trying to reach 100 users per item

In [9]:
allSolution=pd.concat([solution_cbf,solution_int,solution_base],axis=0)
allSolution.shape

(136358, 3)

In [10]:
allSolution[allSolution['item_id']==103]

,item_id,users,dataset
0,103,"968548,1121184,240124,169200,590618,709594,202...",CB
0,103,"2270030,427739,259811,2053817,1948765,242845,3...",INT
0,103,"64851,99374,117614,280728,590206,719974,112857...",BASE


### Let's merge

In [12]:
from  more_itertools import unique_everseen
import time
import pandas as pd
import numpy as np

DATASET_SEQ=['CB','INT']##,'BASE']
DATASET_MIX=[0.35,0.65]##,0.02]

#start timer
t_start = time.time()   

progress=0
### Let's generate results file
with open('jun27_ensemble_solution_35_65.csv','w') as fp:
    for item in sorted(allSolution['item_id'].unique()):
        candidates=[]
        for ind in range(0,len(DATASET_SEQ)):
            #print DATASET_SEQ[ind]
            users=allSolution[(allSolution['item_id']==item) & (allSolution['dataset']==DATASET_SEQ[ind])]['users'].copy()
            users=users.reset_index(drop=True)
            if len(users)>0:
                users=str(users[0]).split(",")
            else:
                users=[]
            #print "users:",len(users)
            candidates=candidates+[(np.float(x[0])/DATASET_MIX[ind],x[1]) for x in sorted(enumerate(users))]
            #print "candidates",len(candidates)
        candidates=[x[1] for x in sorted(candidates)]
        #print len(candidates),len(set(candidates))
        candidates=[x for x in unique_everseen(candidates)][0:100]
        #print len(candidates)                                                                                            
        #print candidates
        item_id=str(item)+"\t"
        fp.write(item_id)
        users=",".join(map(str,candidates))
        fp.write(users)
        fp.write("\n")
        fp.flush()
        progress=progress+1
        if ((progress+1)%5000)==0:
            print "Num items processed so far:",progress
fp.close()

#end timer
t_end=time.time()
print "Time invested "+str((t_end-t_start))+" s"

Num items processed so far: 4999
Num items processed so far: 9999
Num items processed so far: 14999
Num items processed so far: 19999
Num items processed so far: 24999
Num items processed so far: 29999
Num items processed so far: 34999
Num items processed so far: 39999
Num items processed so far: 44999
Time invested 1522.82000017 s
